# Проект - скрапер HTML-страниц

Представим, что нам нужны некоторые данные, которые мы можем получить только с веб-страницы. То есть нет ни таблиц, ни файлов, ни архивов, ни баз данных, где они лежат, а есть только несколько веб-страниц, прямо на которых описаны эти данные. В этом случае нам нужно "заскрапить" (scrape) эти данные. Для этого существует библиотека Beautiful Soup.

## Постановка задачи

Мы напишем не только сам алгоритм скраппинга страницы, но и некоторые дополнительные классы, которые будет удобно использовать для сбора данных. Также продумаем единую точку входа в наш проект, чтобы им можно было удобно пользоваться.

Архитектура модуля будет следующей. Структура файлов:

    scraper
    |- download.py
    |- parse.py
    |- data.py
    |- __init__.py

Таким образом, наш модуль будет разделен на 3 части: файл, в котором описана логика сохранения веб-страницы на локальный копьютер; файл, в котором описан класс-парсер скаченной страницы; а также файл, где будет описана работа с получившимися файлами. Также опишем файл `__init__.py`, в котором будут общие функции, которые помогут использовать весь наш модуль одной строчкой.

### Почему такое странное название файла?

В уроке про модули мы с вами узнали, что любой файл является модулем, который можно импортировать. Например, при такой структуре файлов:

    |- a.py
    |- b.py
    
в файле b.py допустим следующим импорт: `from a import *`, поскольку эти файлы лежат в одной папке. Если же файл a.py лежит в подпапке, т.е. вот так:

    |- a_dir
    |  |- a.py
    |
    |- b.py
    
то логично предположить, что код из него в b.py можно импортировать командой `from a_dir.a import *`, но не тут-то было. Дело в том, что папка в питоне является модулем, если в ней есть файл с названием `__init__.py`, иначе мы не можем взаимодействовать с кодом в этой папке из других модулей. Больше того, код, описанный в этом файле, будет доступен файлов, соседствующих с папкой, через команду `from a_dir import *`, и именно такой формат импорта мы будем считать самым удобным для пользователей нашего модуля.

## Что будем скрапить?

Это вам решать! Найдите любую страницу в интернете, где описаны какие-нибудь данные, и используйте их. В небольших примерах кода будем использовать данные о погоде в Мурманске с сайта [Метеоновости](http://www.hmn.ru/index.php?index=8&value=22113&tz=3&start=2022-11-20&fin=2022-11-28&x=10&y=5).

---

Теперь опишем подробнее, что должно быть реализовано внутри каждого из этих файлов.

## 1. `download.py`

Начнем с файла, в котором описано скачивание веб-страницы на локальный компьютер. Назовем класс-обертку для этого `Downloader`. Суть этого класса в том, что его объект должен содержать четыре вещи:

- адрес страницы, которую мы скрапим;
- параметры, которые мы подаем по этому адресу;
- метод, которым мы обращаемся по адресу, чтобы получить страницу;
- путь к файлу на локальном компьютере, в который будет сохранен исходный код веб-страницы.

Таким образом, объекты этого класса должны иметь возможность следующего использования:

In [ ]:
URL = "http://www.hmn.ru/index.php"      # тут используйте адрес вашего сайта
PARAMS = {                               # эти параметры также индивидуальны для страницы, которую вы скрапите
    "index": 8,
    "value": 22113,
    "tz": 3,
    "start": "2022-11-20",
    "fin": "2022-11-28",
    "x": 10,
    "y": 5,
}
FILE_PATH = "weather.html"               # используйте ваше название. Будет более понятно, если будете исполь-
                                         # зовать расширение html, т.к. в этом файле будет код html-страницы

downloader = Downloader(url=URL, params=PARAMS, method="GET")
downloader.get_html()       # этот метод возвращает строку с контентом, которую получил по запросу на URL
downloader.save(FILE_PATH)  # метод сохраняет полученную строку в файл, путь к которому подается в аргументе

## 2. `parse.py`

После того, как мы скачали файл с исходным кодом веб-страницы, нам нужно выделить из него информацию. Исходный код страницы выглядит примерно так:

    <html><head><title>Мурманск : архив погоды</title>
      <link rel="stylesheet" href="css/styles1.css" type="text/css">
      <link rel="stylesheet" href="css/clicktravel.css" type="text/css"><link rel="stylesheet" href="css/weather.css" type="text/css"> <script async id="__lxGc__rtr" type="text/javascript" src="//s.clickiocdn.com/t/207129/360.js"></script> 
      <script async type='text/javascript' src='//s.luxupcdnc.com/t/common_402.js'></script>
    </head>
    <body bgcolor="white" topmargin="0" marginheight="0">
      
      <center><div id="counters" style="position:absolute; display:none; width:1px; height:1px; z-index:-1"><IMG SRC="http://ww.hmn.ru/counterdb.php?data=NS4yMjguODkuNzI/ISM1LjIyOC44OS43Mj8hI01vemlsbGEvNS4wIChNYWNpbnRvc2g7IEludGVsIE1hYyBPUyBYIDEwLjE1OyBydjo5Ny4wKSBHZWNrby8yMDEwMDEwMSBGaXJlZm94Lzk3LjA/ISM4PyEjaHR0cDovL3d3dy5obW4ucnUvaW5kZXgucGhwP2luZGV4PTgmdmFsdWU9MjIxMTM/ISMwPyEjMD8hIzEwPyEjMD8hIzIyMTEz" width=1 height=1 border=0>        <img src="http://counter.rambler.ru/top100.cnt?319821" alt="Rambler's Top100" width="1" height="1" border="0"></div><table width="790" border="0" cellspacing="0" cellpadding="0" height="38" class="px_blue_fon" style="margin:0px 0px 10px 0px;">
      <tr> 
        <td height="27" > 
        <table border="0" cellspacing="0" cellpadding="3" class="m12" height="20" width="100%">
          <tr> 
            <td align="right" nowrap width="3%"><img src="img/ico_top_travel.gif" width="25" height="25" title="Каталог стран, городов, описание достопримечательностей, климат стран и городов мира, карты стран, поиск попутчиков, туристическая система поиска"></td>
            <td width="5%" nowrap><a href="http://www.svali.ru" class="imp" target="_blank"><b>Туризм</b><br>
            <span class="m11">Свали.ру</span></a></td>
            <td align="right" width="3%" nowrap><img src="img/1px.gif" width="5" height="1"><img src="img/ico_top_avia.gif" width="25" height="25" title="Погода в аэропортах России и мира"></td>
            <td width="19%" nowrap><a href="http://avia.meteonovosti.ru/" class="imp"><b>Погода в аэропортах</b><br>
              <span class="m11">риски задержки</span></a></td>
    ...
    
Его вы можете посмотреть в браузере, щелкнув правой кнопкой мыши на странице и выбрав пункт контекстного меню Исходный код (или исходный код выделенного фрагмента). В Хроме также можно нажать Исследовать элемент.

### Структура веб-страницы

Вся веб-страница состоит из тегов - это слова, обернутые в треугольные скобки, например, `<title>` или `<div align="center">` (во втором случае мы имеем тег с параметром). Эти теги описывают браузеру, как именно нужно отображать веб-страницу. Теги могут быть одинарными и двойными - внутрь блоков, которые описывают вторые, можно поместить какой-то еще контент. Закрывающий тег должен иметь тот же тип, что и открывающий, но начинаться с `/`. Пример тега с контентом и параметрами:

    <td align="right" width="3%">Каталог стран, городов, описание достопримечательностей, климат стран и городов мира, карты стран, поиск попутчиков, туристическая система поиска</td>
    
В приведенном фрагменте описана ячейка таблицы, у которой параметрами заданы ширина и расположение, внутри которой написан текст "Каталог стран...". В подобных тегах и описываеся произвольная веб-страница, и вам нужно найти, в каком именно теге описан фрагмент данных, который нужно сохранить.

Вообще, вся веб-страница делится на два блока: `<head> ... </head>` и `<body> ... </body>`. В первом описываются метаданные страницы, (например, что должно быть написано в названии вкладки и какие файлы со стилями подключать), а во втором располагается уже всё то, что мы видим на самой странице. Поэтому теги, содержащие нужные вам данные, нужно искать внутри блока body.

### Вернемся к заданию

Нужно описать класс Parser, который можно будет использовать следующим образом:

In [ ]:
FILE_PATH = "weather.html"
PARSED_FILE_PATH = "weather.json"


parser = Parser(source=FILE_PATH)    # в конструкторе он принимает путь к файлу, сохраненному Downloader'ом

parser.parse()                       # должен вернуться список или словарь данных, полученных из кода страницы
parser.save(PARSED_FILE_PATH)        # сохраняет данные в виде json- или yaml-файла

Внутри метода parse нужно использовать библиотеку Beautiful Soup. Документация по ней приведена [здесь](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

Разберем небольшой пример, что можно сделать с использованием этой библиотеки в данных о погоде. Соберем данные со страницы http://www.hmn.ru/index.php?index=8&value=22113&tz=3&start=2022-11-20&fin=2022-11-28&x=10&y=5.

![%D0%A1%D0%BD%D0%B8%D0%BC%D0%BE%D0%BA%20%D1%8D%D0%BA%D1%80%D0%B0%D0%BD%D0%B0%202022-11-30%20%D0%B2%2004.03.05.png](attachment:%D0%A1%D0%BD%D0%B8%D0%BC%D0%BE%D0%BA%20%D1%8D%D0%BA%D1%80%D0%B0%D0%BD%D0%B0%202022-11-30%20%D0%B2%2004.03.05.png)

Нажмем на любое значение из таблицы правой кнопкой мыши и выберем Исследовать элемент. Поскольку значений в таблице много, нам ничего не даст использование какого-то конкретного элемента, поэтому найдем тег, в который обернут этот элемент - и в нашем случае это тег table с атрибутом `class="m80"`. Посмотрим, сколько на странице содержится таблиц с таким же классом:

In [ ]:
import bs4
import re
import requests

web_page = requests.get(
    "http://www.hmn.ru/index.php?index=8&value=22113&tz=3&start=2022-11-20&fin=2022-11-28&x=10&y=5",    
).content  # сохранили код веб-страницы в переменную

soup = bs4.BeautifulSoup(web_page, 'html.parser')    # создали объект супа
len(soup.find_all("table", attrs={"class": "m80"}))  # смотрим длину списка таблиц с таким классом

2

Надо выбрать, какая из этих двух таблиц та самая. Пойдем с конца:

In [ ]:
soup.find_all("table", attrs={"class": "m80"})[-1]

<table border="0" cellpadding="0" cellspacing="0" class="m80" style="margin-left:5px;" width="400">
<tr>
<td nowrap="" width="3"><img height="8" src="img/1px.gif" width="3"/></td>
<td class="menu_first_bot" nowrap="" width="1%"><img height="8" src="img/1px.gif" width="6"/></td>
<td class="menu_fon_bot" nowrap="" valign="bottom" width="98%">
<form action="index.php" method="get">
<input name="index" type="hidden" value="8"/>
<input name="value" type="hidden" value="22113"/>
<input name="tz" type="hidden" value="3"/>
<table align="right" border="0" cellpadding="0" cellspacing="0" height="17" width="100%">
<tr>
<td align="center" class="m75" height="30" nowrap="" width="120">
<b class="imp">Выбрать период с </b></td>
<td class="m75" height="21" nowrap=""><select class="m12" name="start"><option value="2022-11-21">пн., ноя. 21</option><option value="2022-11-22">вт., ноя. 22</option><option value="2022-11-23">ср., ноя. 23</option><option value="2022-11-24">чт., ноя. 24</option><option value

Здесь нет наших значений, поэтому будем работать с 0-й таблицей.

In [ ]:
table = soup.find_all("table", attrs={"class": "m80"})[0]

# таблица имеет четкую структуру тегов tr и td, поэтому можем просто индексами выбрать нужную информацию
table.find_all("tr")[:10]

[<tr align="center" class="fon_gray_light">
 <td class="m11" width="15%">Дата и время (местное)</td>
 <td class="m11">Характер<br/>
             погоды</td>
 <td class="m11">Т<br/>
             (<sup>o</sup>C)</td>
 <td class="m11">Тd<br/>
             (<sup>o</sup>C)</td>
 <td class="m11">Отн.<br/>
             влаж.<br/>
             (%)</td>
 <td class="m11">Атм.<br/>
             давл.<br/>
             (мм)</td>
 <td class="m11">Ветер<br/>
             (м/с)</td>
 <td class="m11">Индекс<br/>
 <a href="index.php?index=14&amp;value=8">комф.</a>,<br/>
 <a href="index.php?index=14&amp;value=13">УФИ</a>
 </td>
 <td class="m11" width="6%">Кол-во<br/>
             осад-<br/>
             ков</td>
 <td class="m11">Высота<br/>
             снега (см)</td>
 </tr>,
 <tr>
 <td align="center" class="m11"><b>21:00</b><br/>28.11.2022г.</td>
 <td align="center" class="m11" valign="middle"><img height="50" src="img/signs/n108.gif" title="пасмурно" width="44"/><br/>пасмурно</td>
 <td class="td11 te

In [ ]:
time = []                  # создадим список, где будут храниться все значения времени по порядку в таблице
wind = []                  # а сюда будем добавлять направление ветра

tr = table.tr              # первый тег tr - это заголовок таблицы, он нам не нужен, поэтому сразу переходим к
                           # циклу, где ищем следующий соседний тег <tr>
while tr.find_next_sibling("tr"):  # если следующего тега нет, значит таблица кончилась, можно выходить из цикла
    tr = tr.find_next_sibling("tr")  # теперь tr - это следующий тег <tr> на странице
                  
    if tr.td.b is not None and tr.td["class"][0] == "m11":  # последняя строка таблицы, оказывается, тоже содержит
                                                            # тег <b>, поэтому еще и по классу будем фильтровать
        time.extend(tr.td.b)          # дописываем время в список результатов времени, если мы оказались в той
                                      # строке таблицы, где есть время (а таблица слегка криво построена)
        wind.extend(tr.find_all("td")[7])  # и в том же случае дописываем направление ветра
        
print(*zip(time, temperature), sep="\n")   # выведем результат на экран, объединив два списка в список кортежей

('21:00', 'Ю')
('18:00', 'Ю')
('15:00', 'Ю')
('12:00', 'Ю')
('9:00', 'Ю')
('6:00', 'Ю')
('3:00', 'Ю')
('00:00', 'Ю')
('21:00', 'Ю')
('18:00', 'Ю')
('15:00', 'Ю')
('12:00', 'Ю')
('9:00', 'Ю')
('6:00', 'Ю')
('3:00', 'Ю')
('00:00', 'Ю')
('21:00', 'Ю')
('18:00', 'Ю')
('15:00', 'Ю')
('12:00', 'Ю')
('9:00', 'Ю')
('6:00', 'Ю')
('3:00', 'Ю')
('00:00', 'Ю')
('21:00', 'Ю')
('18:00', 'Ю')
('15:00', 'Ю')
('12:00', 'Ю')
('9:00', 'Ю')
('6:00', 'Ю')
('3:00', 'Ю')
('00:00', 'Ю')
('21:00', 'Ю')
('18:00', 'Ю')
('15:00', 'Ю')
('12:00', 'Ю')
('9:00', 'Ю')
('6:00', 'Ю')
('3:00', 'Ю')
('00:00', 'ЮЗ')
('21:00', 'Ю')
('18:00', 'Ю')
('15:00', 'Ю')
('12:00', 'Ю')
('9:00', 'Ю')
('6:00', 'Ю')
('3:00', 'Ю')
('00:00', 'Ю')
('21:00', 'Ю')
('18:00', 'Ю')
('15:00', 'Ю')
('12:00', 'Ю')
('9:00', 'Ю')
('6:00', 'Ю')
('3:00', 'Ю')
('00:00', 'Ю')
('21:00', 'Ю')
('18:00', 'Ю')
('15:00', 'Ю')
('12:00', 'Ю')
('9:00', 'Ю')


## 3. `data.py`

С собранными и сохраненными данными нужно что-то делать. В этом файле опишите любые классы и функции, которые будут выдавать какие-то осмысленные результаты обработки собранных данных. Обязательно должна быть функция или метод класса, который считывает данные из json или yaml и поставляет их другим функциям/методам в виде питоновских объектов.

## 4. `__init__.py`

Здесь опишем точку входа в наше приложение для потребителя. Опишите функцию process, которая будет принимать на вход адрес страницы, которую мы хотим заскрапить, опциональным аргументом путь к файлу, который будет хранить копию страницы на локальном компьютере, и еще одним опциональным аргументом - путь к файлу, куда мы сохраним результаты парсинга страницы. Возвращать эта функция должна результат исполнения какой-то части логики, описанной в data.py. Также в этом файле должны использоваться сущности из остальных наших файлов, соответственно, их нужно импортировать в начале файла.

In [ ]:
from download import Downloader
from parser import Parser
from data import some_logic

def process(url, web_page_path=None, data_path=None):
    downloader = Downloader(...)
    ...
    parser = Parser(...)
    ...
    data = ...
    return some_logic(data)